In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import cv2
from time import time, sleep
from sklearn.decomposition import IncrementalPCA

from utils import *

### SVD runtime

In [13]:
# SVD comparison on 20s grayscale
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

print('Video loading...')
# Read the video
start = time()
sample_name = 'sample20s_540'
frames, fps, width, height = read_video('data_generated/'+sample_name+'.mp4')
nframes = frames.shape[0]
print(f'Loaded video of {round(nframes/fps, 2)} seconds with quality {width}x{height} in {round(time()-start,2)} seconds')

# Convert to gray scale
print('Transforming the video...')
gray_frames = np.zeros((nframes, height, width), dtype=np.uint8)
for i, frame in enumerate(frames):
    gray_frames[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Fit a SVD model
start = time()
u, s, vh = svds(gray_frames.reshape(gray_frames.shape[0], -1).astype(float), k=50)
after_fit = time()
after_transformations = time()
print(f'Fitting of the scipy SVD model took {round(after_fit-start,2)} seconds')
svd_model = TruncatedSVD(n_components=50)
start = time()
u, s, vh = svd_model.fit(gray_frames.reshape(gray_frames.shape[0], -1))
after_fit = time()
after_transformations = time()
print(f'Fitting of the truncated SVD model took {round(after_fit-start,2)} seconds')
start = time()
u, s, vh = randomized_svd(gray_frames.reshape(gray_frames.shape[0], -1), 50)
after_fit = time()
after_transformations = time()
print(f'Fitting of the randomized SVD model took {round(after_fit-start,2)} seconds')

Video loading...
Loaded video of 20.0 seconds with quality 960x540 in 0.82 seconds
Transforming the video...
Fitting of the scipy SVD model took 20.05 seconds
Fitting of the truncated SVD model took 22.52 seconds
Fitting of the randomized SVD model took 18.98 seconds


Numpy's SVD -> MemoryError: Unable to allocate 1.96 TiB for an array with shape (518400, 518400) and data type float64

### SVD -> PCA

In [104]:
video = VideoLoader('data/sample20s.mp4', duration=60, grayscale=True, scale=(480, 270))

In [110]:
pca_model = IncrementalPCA(n_components=10)
pca_model2 = IncrementalPCA(n_components=10)
all_frames = video.get_all_frames()
original_shape = all_frames.shape
mean, std = np.mean(all_frames), np.std(all_frames)
all_frames_std = (all_frames -  mean) / std #???
frames_after_pca = pca_model.inverse_transform(pca_model.fit_transform(all_frames.reshape(all_frames.shape[0], -1)))
frames_after_pca = frames_after_pca.reshape(all_frames.shape)
frames_after_pca2 = pca_model2.inverse_transform(pca_model2.fit_transform(all_frames_std.reshape(all_frames.shape[0], -1)))
frames_after_pca2 = frames_after_pca2.reshape(all_frames_std.shape)
frames_after_pca2 = (frames_after_pca2*std) + mean
write_video('data/20pca.mp4', frames_after_pca, all_frames.shape[2], all_frames.shape[1], 24, grayscale=True)
write_video('data/20pca_std.mp4', frames_after_pca2, all_frames.shape[2], all_frames.shape[1], 24, grayscale=True)

In [111]:
from sklearn.utils.extmath import randomized_svd

frames_svd = all_frames_std.reshape(all_frames.shape[0], -1)

u, _, _ = randomized_svd(frames_svd.T, 10)
proj = u.T @ frames_svd.T

after_proj = (u @ proj).T
after_proj = after_proj.reshape(all_frames.shape)
after_proj = (after_proj*std)+mean
write_video('data/20svd.mp4', after_proj, original_shape[2], original_shape[1], 24, grayscale=True)

In [112]:
(reconstruction_error(np.clip(frames_after_pca, 0, 255), all_frames), 
 reconstruction_error(np.clip(frames_after_pca2, 0, 255), all_frames), 
 reconstruction_error(np.clip(after_proj, 0, 255), all_frames))

(27.96268172632351, 27.9626817263235, 34.95518856158816)

### PCA model using SVD

In [19]:
video = VideoLoader('data/sample20s.mp4', duration=60, grayscale=True, scale=(480, 270))

frames = video.get_all_frames()

pca_model = custom_pca(ncomp=10)
pca_model.fit(frames)
reduced = pca_model.transform(frames)

reconstructed = pca_model.inverse_transform(reduced).reshape(frames.shape)
reconstruction_error(frames, np.clip(reconstructed,0,255))

34.95518856158816

In [16]:
show_video(video.get_all_frames())

### Random video loader

In [25]:
v = read_video('data/sample20s.mp4', nframes=24*3)
vr = read_video('data/sample20s.mp4', nframes=24*3, random=True)

In [26]:
v[0].shape, vr[0].shape

(72, 720, 1280, 3)

In [31]:
show_video(v[0])
show_video(vr[0])

### SVD on long video, comparison of reconstruction error

In [2]:
frames, meta = read_video('data_generated/sample20s_540.mp4')
original_shape = frames.shape
pca_svd = custom_pca()
t1 = time()
pca_svd.fit(frames.reshape(frames.shape[0], -1))
t2 = time()
print('Fitting time:', t2-t1)
new_frames = pca_svd.inverse_transform(pca_svd.transform(frames)).reshape(original_shape)
t3 = time()
print('Transform:', t3-t2)

#write_video('data_generated/sample20s_540_svdpca.mp4', new_frames, meta['width'], meta['height'], meta['fps'])
print(reconstruction_error(frames.reshape(new_frames.shape), new_frames))

Fitting time: 64.06482172012329
Transform: 41.710843086242676
8.52525261354381


In [16]:
total_frames = original_shape[0]
nframes = int(total_frames*0.5)
sample_frames, meta = read_video('data_generated/sample20s_540.mp4', nframes=nframes, random=True)
pca_svd = custom_pca()
t1 = time()
pca_svd.fit(sample_frames.reshape(nframes, -1))
t2 = time()
print('Fitting time:', t2-t1)

new_frames = np.zeros(original_shape)
for i, batch_frames in enumerate(VideoLoader('data_generated/sample20s_540.mp4')):
    new_batch = pca_svd.inverse_transform(pca_svd.transform(batch_frames))
    new_frames[i*64:(i+1)*64] = new_batch.reshape(batch_frames.shape)
t3 = time()
print('Transform:', t3-t2)

#write_video('data_generated/sample20s_540_0.5.mp4', new_frames, meta['width'], meta['height'], meta['fps'])
print(reconstruction_error(frames.reshape(new_frames.shape), new_frames))

Fitting time: 36.644392251968384
Transform: 15.273186206817627
41.60200538114069


In [3]:
total_frames = original_shape[0]
nframes = int(total_frames*0.25)
sample_frames, meta = read_video('data_generated/sample20s_540.mp4', nframes=nframes, random=True)
pca_svd = custom_pca()
t1 = time()
pca_svd.fit(sample_frames.reshape(nframes, -1))
t2 = time()
print('Fitting time:', t2-t1)

new_frames = np.zeros(original_shape)
for i, batch_frames in enumerate(VideoLoader('data_generated/sample20s_540.mp4')):
    new_batch = pca_svd.inverse_transform(pca_svd.transform(batch_frames))
    new_frames[i*64:(i+1)*64] = new_batch.reshape(batch_frames.shape)
t3 = time()
print('Transform:', t3-t2)

write_video('data_generated/sample20s_540_0.25.mp4', new_frames, meta['width'], meta['height'], meta['fps'])
print(reconstruction_error(frames.reshape(new_frames.shape), new_frames))

Fitting time: 12.123085021972656
Transform: 15.299276351928711
48.23870056503986


In [8]:
total_frames = original_shape[0]
nframes = int(total_frames*0.1)
sample_frames, meta = read_video('data_generated/sample20s_540.mp4', nframes=nframes, random=True)
pca_svd = custom_pca()
t1 = time()
pca_svd.fit(sample_frames.reshape(nframes, -1))
t2 = time()
print('Fitting time:', t2-t1)

new_frames = np.zeros(original_shape)
for i, batch_frames in enumerate(VideoLoader('data_generated/sample20s_540.mp4')):
    new_batch = pca_svd.inverse_transform(pca_svd.transform(batch_frames))
    new_frames[i*64:(i+1)*64] = new_batch.reshape(batch_frames.shape)
t3 = time()
print('Transform:', t3-t2)

write_video('data_generated/sample20s_540_0.1.mp4', new_frames, meta['width'], meta['height'], meta['fps'])
print(reconstruction_error(frames.reshape(new_frames.shape), new_frames))

Fitting time: 6.648501873016357
Transform: 15.267206192016602
49.19006924618484


In [10]:
total_frames = original_shape[0]
nframes = int(total_frames*0.05)
sample_frames, meta = read_video('data_generated/sample20s_540.mp4', nframes=nframes, random=True)
pca_svd = custom_pca()
t1 = time()
pca_svd.fit(sample_frames.reshape(nframes, -1))
t2 = time()
print('Fitting time:', t2-t1)

new_frames = np.zeros(original_shape)
for i, batch_frames in enumerate(VideoLoader('data_generated/sample20s_540.mp4')):
    new_batch = pca_svd.inverse_transform(pca_svd.transform(batch_frames))
    new_frames[i*64:(i+1)*64] = new_batch.reshape(batch_frames.shape)
t3 = time()
print('Transform:', t3-t2)

write_video('data_generated/sample20s_540_0.05.mp4', new_frames, meta['width'], meta['height'], meta['fps'])
print(reconstruction_error(frames.reshape(new_frames.shape), new_frames))

Fitting time: 5.889437913894653
Transform: 15.425113677978516
271.09038918761786


### Comparison of reconstruction with PCA

In [2]:
video = VideoLoader('data_generated/sample20s_540.mp4')

incremental_pca = IncrementalPCA(n_components=10)
t1 = time()
for frames in video:
    incremental_pca.partial_fit(frames.reshape(frames.shape[0], -1))
t2 = time()
reduced_frames = incremental_pca.transform(video.get_all_frames().reshape(video.total_frames, -1))
new_frames = incremental_pca.inverse_transform(reduced_frames).reshape(video.total_frames, video.height, video.width, 3)
t3 = time()

print('Fitting time:', t2-t1)
print('Transform:', t3-t2)

print(reconstruction_error(video.get_all_frames(), new_frames))

Fitting time: 331.5185239315033
Transform: 32.415708780288696
32.78112737868529


In [2]:
video = VideoLoader('data_generated/sample20s_540.mp4', batch_size=96)

incremental_pca = IncrementalPCA(n_components=10)
t1 = time()
for frames in video:
    incremental_pca.partial_fit(frames.reshape(frames.shape[0], -1))
t2 = time()
reduced_frames = incremental_pca.transform(video.get_all_frames().reshape(video.total_frames, -1))
new_frames = incremental_pca.inverse_transform(reduced_frames).reshape(video.total_frames, video.height, video.width, 3)
t3 = time()

print('Fitting time:', t2-t1)
print('Transform:', t3-t2)

print(reconstruction_error(video.get_all_frames(), new_frames))

Fitting time: 382.18158316612244
Transform: 38.59532570838928
32.781283160921156


### Big sample

In [ ]:
model = custom_pca()

video = VideoLoader('data/big_sample.mp4', grayscale=True, scale=(256, 256))
t1 = time()
frames_rand = video.get_random_frames(0.2)
model.fit(frames_rand)
t2 = time()
full_shape = (video.total_frames, video.height, video.width)
reconstructed = np.zeros(full_shape)
for j, frames in enumerate(video):
    nbatch = frames.shape[0]
    reconstructed[j*nbatch:(j+1)*nbatch] = model.inverse_transform(model.transform(frames)).reshape(nbatch, video.width, video.height)
t3 = time()
print(reconstruction_error(video.get_all_frames(), reconstructed))
t4 = time()

print('Fitting time:', t2-t1)
print('Transform:', t3-t2)
print('Error calculation:', t4-t3)

write_video('data/big_sample_reconstructed.mp4', reconstructed, video.width, video.height, video.fps, grayscale=True)

608.3557539173149
Fitting time: 46.197591066360474
Transform: 87.10507035255432
Error calculation: 88.43285274505615
